Dataset Generation

In [1]:
from numpy import zeros, dot, conj, prod, sqrt, exp, pi, diag, angle, array, argwhere, real, floor, frombuffer, uint8, where, stack, asarray
from numpy.linalg import qr, multi_dot, svd
from numpy.random import uniform, normal, randint
import matplotlib.pyplot as plt
import pandas as pd
import io
import cv2
import pickle

In [2]:
class Node:
    def __init__(self, pls_params):
        """
        Initialization of class
        :param pls_params: object from PLSParameters class containing basic parameters
        """
        self.bandwidth = pls_params.bandwidth
        self.bin_spacing = pls_params.bin_spacing
        self.num_ant = pls_params.num_ant
        self.bit_codebook = pls_params.bit_codebook

        self.NFFT = pls_params.NFFT
        self.num_used_bins = pls_params.num_used_bins
        self.subband_size = self.num_ant

        self.num_subbands = pls_params.num_subbands
        self.num_PMI = self.num_subbands

        self.key_len = self.num_subbands * self.bit_codebook

    def unitary_gen(self):
        """
        Generate random nitary matrices for each sub-band
        :return GA: Unitary matrices in each sub-band at Alice
        """
        GA = zeros(self.num_subbands, dtype=object)
        for sb in range(0, self.num_subbands):
            Q, R = qr(uniform(0, 1, (self.num_ant, self.num_ant))
                      +1j*uniform(0, 1, (self.num_ant, self.num_ant)))

            GA[sb] = dot(Q, diag(diag(R)/abs(diag(R))))
        return GA

    @staticmethod
    def awgn(in_signal, SNRdB):
        """
        Adds AWGN to the input signal. Maintains a given SNR.
        :param in_signal: input signal to which noise needs to be addded
        :param SNRdB: Signal to Noise Ratio in dB
        :return: noisy signal
        """
        S0 = in_signal*conj(in_signal)
        S = S0.sum() / prod(in_signal.shape)
        SNR = 10 ** (SNRdB / 10)
        N = S.real / SNR
        awg_noise = sqrt(N / 2) * normal(0, 1, in_signal.shape) + \
                    1j * sqrt(N / 2) * normal(0, 1, in_signal.shape)

        return in_signal + awg_noise

    def sv_decomp(self, rx_sig):
        """
        Perform SVD for the matrix in each sub-band
        :param rx_sig: Channel matrix at the receiver in each sub-band
        :return lsv, sval, rsv: Left, Right Singular Vectors and Singular Values for the matrix in each sub-band
        """
        lsv = zeros(self.num_subbands, dtype=object)
        sval = zeros(self.num_subbands, dtype=object)
        rsv = zeros(self.num_subbands, dtype=object)

        for sb in range(0, self.num_subbands):
            U, S, VH = svd(rx_sig[sb])
            V = conj(VH).T
            ph_shift_u = diag(exp(-1j * angle(U[0, :])))
            ph_shift_v = diag(exp(-1j * angle(V[0, :])))
            lsv[sb] = dot(U, ph_shift_u)
            sval[sb] = S
            rsv[sb] = dot(V, ph_shift_v)

        return lsv, sval, rsv

    def receive(self, *args):
        """
        Contains 3 cases for the 3 steps of the process depending who is the receiver (Alice or Bob)
        Generates the frequency domain rx signal in each sub-band which is of the form H*G*F
        H - channel, G - random unitary or LSV from SVD, F - DFT precoder
        :param args: 0 - who is receiving, 1 - signal to noise ratio in dB, 2 - freq domain channel,
        3 - random unitary or LSV from SVD, 4 - DFT precoder
        :return: frequency domain rx signal in each sub-band
        """
        rx_node = args[0]
        SNRdB = args[1]
        if rx_node == 'Bob' and len(args) == 4:
                HAB = args[2]
                GA = args[3]
                rx_sigB = zeros(self.num_subbands, dtype=object)
                for sb in range(0, self.num_subbands):
                    tx_sig = multi_dot([HAB[sb], GA[sb]])
                    # rx_sigB[sb] = tx_sig
                    rx_sigB[sb] = self.awgn(tx_sig, SNRdB)
                return rx_sigB
        if rx_node == 'Bob' and len(args) == 5:
            HAB = args[2]
            UA = args[3]
            FA = args[4]
            rx_sigB = zeros(self.num_subbands, dtype=object)
            for sb in range(0, self.num_subbands):
                tx_sig = multi_dot([HAB[sb], conj(UA[sb]), conj(FA[sb]).T])
                rx_sigB[sb] = self.awgn(tx_sig, SNRdB)
            return rx_sigB
        elif rx_node == 'Alice' and len(args) == 5:
            HBA = args[2]
            UB = args[3]
            FB = args[4]
            rx_sigA = zeros(self.num_subbands, dtype=object)
            for sb in range(0, self.num_subbands):
                tx_sig = multi_dot([HBA[sb], conj(UB[sb]), conj(FB[sb]).T])
                rx_sigA[sb] = self.awgn(tx_sig, SNRdB)
                # rx_sigA[sb] = tx_sig
            return rx_sigA


    def secret_key_gen(self):
        """
        Generate private info bits in each sub-band
        :return bits_subband: private info bits in each sub-band
        """
        bits_subband = zeros(self.num_subbands, dtype=object)

        secret_key = randint(0, 2, self.key_len)

        # Map secret key to subbands
        for sb in range(self.num_subbands):
            start = sb * self.bit_codebook
            fin = start + self.bit_codebook

            bits_subband[sb] = secret_key[start: fin]

        return bits_subband

    def precoder_select(self, bits_subband, codebook):
        """
        selects the DFT precoder from the DFT codebook based. Bits are converted to decimal and used as look up index.
        :param bits_subband: Bits in each sub-band
        :param codebook: DFT codebook of matrix precoders
        :return precoder: Selected DFT preocder from codebook for each sub-band
        """
        precoder = zeros(self.num_subbands, dtype=object)

        for sb in range(self.num_subbands):
            bits = bits_subband[sb]
            start = self.bit_codebook - 1
            bi2dec_wts = 2**(array(range(start, -1, -1)))
            codebook_index = sum(bits*bi2dec_wts)
            precoder[sb] = codebook[codebook_index]

        return precoder
    @staticmethod
    def dec2binary(x, num_bits):
        """
        Covert decimal number to binary array of ints (1s and 0s)
        :param x: input decimal number
        :param num_bits: Number bits required in the binary format
        :return bits: binary array of ints (1s and 0s)
        """
        bit_str = [char for char in format(x[0, 0], '0' + str(num_bits) + 'b')]
        bits = array([int(char) for char in bit_str])
        # print(x[0, 0], bits)
        return bits

    def PMI_estimate(self, rx_precoder, codebook):
        """
        Apply minumum distance to estimate the transmitted precoder, its index in the codebook and the binary equivalent
        of the index
        :param rx_precoder: observed precoder (RSV of SVD)
        :param codebook: DFT codebook of matrix precoders
        :return PMI_sb_estimate, bits_sb_estimate: Preocder matrix index and bits for each sub-band
        """
        PMI_sb_estimate = zeros(self.num_subbands, dtype=int)
        bits_sb_estimate = zeros(self.num_subbands, dtype=object)

        for sb in range(self.num_subbands):
            dist = zeros(len(codebook), dtype=float)

            for prec in range(len(codebook)):
                diff = rx_precoder[sb] - codebook[prec]
                diff_squared = real(diff*conj(diff))
                dist[prec] = sqrt(diff_squared.sum())
            min_dist = min(dist)
            PMI_estimate = argwhere(dist == min_dist)
            PMI_sb_estimate[sb] = PMI_estimate
            bits_sb_estimate[sb] = self.dec2binary(PMI_estimate, self.bit_codebook)

        return PMI_sb_estimate, bits_sb_estimate

In [5]:
class PLSParameters:

    def __init__(self, prof):
        """
        Initialization of class
        :param prof: PLS profile containing basic parameters such as bandwidth, antennas bin spacing, bits in codebook
        index
        """
        self.bandwidth = prof['bandwidth']
        self.bin_spacing = prof['bin_spacing']
        self.num_ant = prof['num_ant']
        self.bit_codebook = prof['bit_codebook']

        self.NFFT = int(floor(self.bandwidth/self.bin_spacing))
        self.num_used_bins = self.NFFT - 2
        self.subband_size = self.num_ant

        self.num_subbands = int(floor(self.num_used_bins/self.subband_size))
        self.num_PMI = self.num_subbands

    def codebook_gen(self):
        """
        Generate DFT codebbok of matrix preocders
        :return: matrix of matrix preocders
        """
        num_precoders = 2**self.bit_codebook
        codebook = zeros(num_precoders, dtype=object)

        for p in range(0, num_precoders):
            precoder = zeros((self.num_ant, self.num_ant), dtype=complex)
            for m in range(0, self.num_ant):
                for n in range(0, self.num_ant):
                    w = exp(1j*2*pi*(n/self.num_ant)*(m + p/num_precoders))
                    precoder[n, m] = (1/sqrt(self.num_ant))*w

            codebook[p] = precoder

        return codebook

    def channel_gen(self):
        """
        Generate generic Rayleigh fading channels in the frequency domain
        :return:
        """
        HAB = zeros(self.num_subbands, dtype=object)
        HBA = zeros(self.num_subbands, dtype=object)

        for sb in range(0, self.num_subbands):
            H = 1/sqrt(2)*(normal(0, 1, (self.num_ant, self.num_ant)) + 1j*normal(0, 1, (self.num_ant, self.num_ant)))
            HAB[sb] = H
            HBA[sb] = H.T

        return HAB, HBA


In [6]:
def bin_array2dec(bin_array):
    arr_reversed = bin_array[::-1]
    dec = 0
    for j in range(len(arr_reversed)):
        dec += (2 ** j) * arr_reversed[j]
    return dec

def crop_center(img,cropx,cropy):
    y,x = img.shape
    startx = x//2-(cropx//2)
    starty = y//2-(cropy//2)    
    return img[starty:starty+cropy,startx:startx+cropx]

def get_img_from_fig(fig, dpi=180):
    buf = io.BytesIO()
    fig.savefig(buf, format="png", dpi=dpi)
    buf.seek(0)
    img_arr = frombuffer(buf.getvalue(), dtype=uint8)
    buf.close()
    img = cv2.imdecode(img_arr, 1)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    scaled_image = cv2.resize(img, (100, 100))  
#     print(scaled_image.shape)
    
    cropped_img = crop_center(scaled_image, 70, 70)
    cropped_img = cv2.bitwise_not(cropped_img)
    return cropped_img

In [7]:
max_SNR = 20
# SNR_dB = range(0, max_SNR, 5)
SNR_dB = [1000]
# SNR_dB = [45, 45]
max_iter = 10

pls_profiles = {
               0: {'bandwidth': 960e3, 'bin_spacing': 15e3, 'num_ant': 2, 'bit_codebook': 2},
               }

# dbg = 1
# for prof in pls_profiles.keys():
#     df = pd.DataFrame(list(pls_profiles[prof].items()),columns = ['column 1', 'column 2'])
# dbg = 1
df = pd.DataFrame(columns = ['Bandwidth', 'Bin Spacing', 'Antennas',
                             'Bit Codebook', 'SNR', 'Obs Precoder', 'Correct PMI'])
dbg = 1

for prof in pls_profiles.values():
    pls_params = PLSParameters(prof)
    codebook = pls_params.codebook_gen()
    
    N = Node(pls_params)  # Wireless network node - could be Alice or Bob

    for s in range(len(SNR_dB)):
 
        for i in range(max_iter):
            HAB, HBA = pls_params.channel_gen()

            ## 1. Alice to Bob
            GA = N.unitary_gen()
            rx_sigB0 = N.receive('Bob', SNR_dB[s], HAB, GA)

            ## 1. At Bob
            UB0 = N.sv_decomp(rx_sigB0)[0]
            bits_subbandB = N.secret_key_gen()
            transmitted_PMI = bin_array2dec(bits_subbandB[0])

            FB = N.precoder_select(bits_subbandB, codebook)

            ## 2. Bob to Alice
            rx_sigA = N.receive('Alice', SNR_dB[s], HBA, UB0, FB)

            ## 2. At Alice
            UA, _, VA = N.sv_decomp(rx_sigA)
            


            fig = plt.figure()
            ax = fig.add_subplot(111)
            ax.plot(VA[0].real, VA[0].imag, 'o', color='black')
            plt.xlim((-1, 1))
            plt.ylim((-1, 1))
            plt.close(fig)
            plot_img_np = get_img_from_fig(fig)
#             plt.imshow(plot_img_np)
#             plt.show()
            df = df.append({'Bandwidth': pls_params.bandwidth,
                            'Bin Spacing': pls_params.bin_spacing,
                            'Antennas': pls_params.num_ant,
                             'Bit Codebook': pls_params.bit_codebook,
                            'SNR': SNR_dB[s],
                            'Obs Precoder': plot_img_np,
                            'Correct PMI': transmitted_PMI},
                           ignore_index=True)
    
#             print(plot_img_np.shape)
        
            dbg = 1


dbg = 1


num_images = df.shape[0]
img_arrays = [df['Obs Precoder'][i] for i in range(num_images)]
precoder_img_data = stack(img_arrays, axis=0)
precoder_labels = array(df['Correct PMI'].tolist())
# print(precoder_img_data.shape)

# for i in range(num_images):
#     plt.imshow(precoder_img_data[i, :, :], cmap = plt.cm.gray)
#     plt.show()

# print('Done')

In [8]:
# print(df.head())
# print(df['Obs Precoder'][0])
# print(df['Obs Precoder'][5].shape)
# plt.imshow(df['Obs Precoder'][5],  cmap = plt.cm.gray)

In [9]:
import torch 
from torch.utils.data import Dataset, DataLoader, random_split

In [10]:
class PrecoderDataset(Dataset):
    def __init__(self, data, target, transform=None):
        self.data = torch.from_numpy(data).float()
        self.target = torch.from_numpy(target).long()
        self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        
        if self.transform:
            x = self.transform(x)
        return x, y
    def __len__(self):
        return len(self.data)

precoder_dataset = PrecoderDataset(precoder_img_data, precoder_labels)
train, val = random_split(precoder_dataset, [int(0.8*max_iter), int(0.2*max_iter)])
train_loader = DataLoader(
    train,
    batch_size=100,
    shuffle=True,
    num_workers=0,
)
# increase batch size
val_loader = DataLoader(
    val,
    batch_size=100,
    shuffle=True,
    num_workers=0,
)

In [11]:
for batch_idx, (data, target) in enumerate(train_loader):
    print('Train: Batch idx {}, data shape {}, target shape {}'.format(
        batch_idx, data.shape, target.shape))
    
for batch_idx, (data, target) in enumerate(val_loader):
    print('Val: Batch idx {}, data shape {}, target shape {}'.format(
        batch_idx, data.shape, target.shape))

Train: Batch idx 0, data shape torch.Size([8, 70, 70]), target shape torch.Size([8])
Val: Batch idx 0, data shape torch.Size([2, 70, 70]), target shape torch.Size([2])


In [12]:
from torch import nn 
from torch import optim 
from torchsummary import summary
import torch.nn.functional as F

In [13]:
case = 0 # training for the first pls profile
num_classes = 2**pls_profiles[case]['bit_codebook']
print(num_classes)

4


In [14]:
# model = nn.Sequential(
#         nn.Linear(70*70, 2000),
#         nn.ReLU(),
#         nn.Linear(2000, 1000),
#         nn.ReLU(),
#         nn.Linear(1000, 64),
#         nn.ReLU(),
#         nn.Linear(64, num_classes),
#         nn.Softmax(),
# )

model = nn.Sequential(
        nn.Linear(70*70, 1000),
        nn.ReLU(),
        nn.Linear(1000, 64),
        nn.ReLU(),
        nn.Linear(64, num_classes),
        nn.Softmax(),
)

In [15]:
# use_cuda = torch.cuda.is_available()

# if use_cuda:
#     model.cuda()

# summary(model, (70, 70))

In [16]:
optimizer = optim.SGD(model.parameters(), lr=1e-2)

In [17]:
loss = nn.CrossEntropyLoss()

In [18]:
num_epochs = 500 # Number of times you go through the whole dataset
for epoch in range(num_epochs):
    losses = list()
    for batch in train_loader:
        
        x, y = batch # extracted from the batch 
        
        # x: batch size x 1 channel x height x width 
        batch_size = x.size(0)

        x = x.view(batch_size, -1) # -1 means it will mutiply all the dimensions that are not the batch size
        # no the size of x will be batch_size rows and height*width columns
#         print(x.size())
        # step 1: forward pass 
        prob = model(x) 
        
#         print(prob)
#         print(y)
        # step 2: compute objective function - measuring distance between the output of the network vs actual answer 
        obj_func = loss(prob, y)
        
        # step 3: clear the gradients 
#         model.zero_grad()
        
        # step 4: accumulate partial derivatives of obj_func wrt parameters 
        obj_func.backward()
        
        # step 5: step in the opposite direction of the gradient 
        optimizer.step()
#         print(obj_func.item())
        losses.append(obj_func.item())
    print(f'Epoch {epoch + 1}, training loss: {torch.tensor(losses).mean():.2f}')
    
    

/home/utsa/anaconda3/envs/py37/lib/python3.7/site-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch 1, training loss: 1.35
Epoch 2, training loss: 0.89
Epoch 3, training loss: 0.87
Epoch 4, training loss: 0.87
Epoch 5, training loss: 0.87
Epoch 6, training loss: 0.87
Epoch 7, training loss: 0.87
Epoch 8, training loss: 0.87
Epoch 9, training loss: 0.87
Epoch 10, training loss: 0.87
Epoch 11, training loss: 0.87
Epoch 12, training loss: 0.87
Epoch 13, training loss: 0.87
Epoch 14, training loss: 0.87
Epoch 15, training loss: 0.87
Epoch 16, training loss: 0.87
Epoch 17, training loss: 0.87
Epoch 18, training loss: 0.87
Epoch 19, training loss: 0.87
Epoch 20, training loss: 0.87
Epoch 21, training loss: 0.87
Epoch 22, training loss: 0.87
Epoch 23, training loss: 0.87
Epoch 24, training loss: 0.87
Epoch 25, training loss: 0.87
Epoch 26, training loss: 0.87
Epoch 27, training loss: 0.87
Epoch 28, training loss: 0.87
Epoch 29, training loss: 0.87
Epoch 30, training loss: 0.87
Epoch 31, training loss: 0.87
Epoch 32, training loss: 0.87
Epoch 33, training loss: 0.87
Epoch 34, training 